# Embedding Clustering & Visualization Pipeline

This notebook processes an input file of vector embeddings, applies dimensionality reduction and clustering, and produces a 2D projection suitable for interactive visualization (e.g., in Streamlit). It also supports hyperparameter tuning of the clustering step to help identify optimal parameters.

---

## Requirements

- **Input file format**:  
  A Parquet file (`.parquet`) with at least:  
  - `file` – string identifier for each row (e.g., filename)  
  - `embedding` – list of floats of equal length (e.g., 750D vectors)  

- **Dependencies**:  
  - `numpy`, `pandas`, `tqdm`  
  - `scikit-learn` (`PCA`, `normalize`, `silhouette_score`)  
  - `hdbscan`  
  - `umap-learn`  

---

## Processing Steps

1. **Load embeddings**  
   - Reads parquet file into a DataFrame.  
   - Ensures embeddings are equal-length vectors.  

2. **PCA reduction (denoising)**  
   - Optionally L2-normalizes embeddings.  
   - Reduces dimensionality (e.g., 750 → 300).  
   - Adds a new column: `embedding_pca`.  

3. **Clustering with HDBSCAN**  
   - L2-normalizes vectors (so Euclidean ≈ Cosine).  
   - Runs HDBSCAN to assign clusters.  
   - Outputs:  
     - `cluster` – cluster ID per point (-1 = noise)  
     - `probability` – confidence score  

4. **Hyperparameter tuning (optional)**  
   - Runs clustering across a parameter grid:  
     - `min_cluster_size`  
     - `min_samples`  
     - `cluster_selection_epsilon`  
   - Collects metrics for each run:  
     - Silhouette score (with/without noise)  
     - Number of clusters  
     - % of points clustered  
   - Results are stored in `df_grid`, sorted by silhouette score.  

5. **UMAP reduction (visualization)**  
   - Reduces embeddings to 2D (`dim1`, `dim2`).  
   - Uses normalized vectors when available.  

6. **Save results**  
   - Final DataFrame includes:  
     - PCA embeddings (`embedding_pca`)  
     - Clusters & probabilities  
     - UMAP coordinates (`dim1`, `dim2`)  
   - Written to parquet: `embeddings_result.parquet`  

This output can be directly consumed by a Streamlit app for interactive visualization.


In [2]:
import itertools
import numpy as np
import pandas as pd
from tqdm import tqdm 

from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize
from sklearn.metrics import silhouette_score

import hdbscan
import umap


In [3]:
def load_embeddings(parquet_path: str) -> pd.DataFrame:
    """
    Load embeddings parquet file into a DataFrame.
    Expects columns: 'file' (str), 'embedding' (list of floats).
    """
    df = pd.read_parquet(parquet_path)
    print(f"✅ Loaded {len(df):,} rows from {parquet_path}")
    print("Columns:", df.columns.tolist())
    return df

# ---------- Helpers ----------
def _column_of_lists_to_matrix(df: pd.DataFrame, col: str) -> np.ndarray:
    """Safely stack a df column of equal-length lists/arrays into a 2D float32 matrix."""
    if col not in df.columns:
        raise ValueError(f"Column '{col}' not found in DataFrame.")
    # Convert to ndarray
    arrs = df[col].to_numpy()
    # Ensure each entry is array-like and same length
    first_len = len(arrs[0])
    if not all(hasattr(x, "__len__") and len(x) == first_len for x in arrs):
        raise ValueError(f"Column '{col}' must contain equal-length vectors.")
    X = np.vstack(arrs).astype(np.float32, copy=False)
    return X

def _matrix_to_list_column(X: np.ndarray) -> list:
    """Convert 2D matrix (N, D) to list-of-lists for storing back in a df column."""
    return [row.astype(float).tolist() for row in X]


# ---------- 1) PCA: 750D -> 300D ----------
def pca_reduce(
    df: pd.DataFrame,
    source_col: str = "embedding",
    target_col: str = "embedding_pca",
    n_components: int = 300,
    l2_normalize: bool = True,
    random_state: int = 42
) -> pd.DataFrame:
    """
    Apply optional L2-normalization (row-wise) then PCA to 'n_components'.
    Adds/updates df[target_col] with list-of-floats length n_components.
    """
    X = _column_of_lists_to_matrix(df, source_col)
    if l2_normalize:
        X = normalize(X, norm="l2")

    pca = PCA(n_components=n_components, random_state=random_state)
    X_pca = pca.fit_transform(X)
    df[target_col] = _matrix_to_list_column(X_pca)
    return df


# ---------- 2) Clustering: HDBSCAN ----------

def cluster_hdbscan_silhouette(
    df: pd.DataFrame,
    source_col: str = "embedding_pca",      # or "embedding"
    min_cluster_size: int = 10,             # 30
    metric: str = "cosine",                 # kept for API compatibility; ignored (we use euclidean after L2 norm)
    l2_normalize: bool = True,
    cluster_selection_epsilon: float = 0.0,
    min_samples: int = 5,
    normalized_col: str = "embedding_norm"
    
) -> float:
    """
    Run HDBSCAN clustering and return silhouette score.
    - L2-normalizes vectors (so Euclidean ≡ Cosine).
    - Returns silhouette score (higher = better separation).
      If clustering produces only 1 cluster or only noise, returns None.
    """

    # stack to (N, D)
    X = _column_of_lists_to_matrix(df, source_col)

    # L2-normalize (row-wise) to make euclidean ≡ cosine
    if l2_normalize:
        Xn = normalize(X, norm="l2")
    else:
        Xn = X

    # Save normalized vectors back to df
    df[normalized_col] = [row.astype(float).tolist() for row in Xn]

    # Run HDBSCAN
    clusterer = hdbscan.HDBSCAN(
        min_cluster_size=min_cluster_size,
        min_samples=min_samples,
        metric="euclidean",  # force euclidean after L2 norm
        cluster_selection_epsilon=cluster_selection_epsilon
    )
    labels = clusterer.fit_predict(Xn)

    # Count clusters and % clustered
    n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
    mask = labels != -1
    pct_clustered = float(mask.mean() * 100.0)
    
    # silhouette requires at least 2 clusters (ignores noise as -1)
    n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
    if n_clusters <= 1:
        return None  # silhouette score undefined in this case
    
    mask = labels != -1
    if mask.sum() == 0:
        return None  # no clustered points
    unique_clusters = set(labels[mask])
    if len(unique_clusters) <= 1:
        return None  # silhouette requires at least 2 clusters
    
    score_all = silhouette_score(Xn, labels, metric="euclidean")
    score = silhouette_score(Xn[mask], labels[mask], metric="euclidean")

    return score_all,score,n_clusters, pct_clustered


def cluster_hdbscan(
    df: pd.DataFrame,
    source_col: str = "embedding_pca",      # or "embedding"
    target_col: str = "cluster",
    min_cluster_size: int = 10, #30
    metric: str = "cosine",                  # kept for API compatibility; ignored (we use euclidean after L2 norm)
    l2_normalize: bool = True,
    cluster_selection_epsilon: float = 0.0,
    min_samples: int =5,
    random_state: int = 42,
    normalized_col: str = "embedding_norm",   # NEW: where to store normalized vectors
    prob_col: str = "probability"
) -> pd.DataFrame:
    """
    Cosine-equivalent HDBSCAN: L2-normalize vectors, then cluster with Euclidean.
    - Adds/updates df[normalized_col] with the normalized vectors (list per row).
    - Adds/updates df[target_col] with integer labels (-1 = noise).
    """
    # stack to (N, D)
    X = _column_of_lists_to_matrix(df, source_col)

    # L2-normalize (row-wise) to make euclidean ≡ cosine
    if l2_normalize:
        Xn = normalize(X, norm="l2")
    else:
        Xn = X  # not recommended for cosine equivalence

    # Save normalized vectors back to df (as lists) so you can reuse without recomputing
    df[normalized_col] = [row.astype(float).tolist() for row in Xn]

    # HDBSCAN with Euclidean on normalized vectors (cosine-equivalent)
    clusterer = hdbscan.HDBSCAN(
        min_cluster_size=min_cluster_size,
        min_samples=min_samples,
        metric="euclidean",                   # force euclidean after L2 norm
        cluster_selection_epsilon=cluster_selection_epsilon
    )
    labels = clusterer.fit_predict(Xn)
    df[target_col] = labels.astype(int)
    df[prob_col] = clusterer.probabilities_.astype(float)
    return df

# ---------- 3) 2D UMAP for visualization ----------
def umap_reduce(
    df: pd.DataFrame,
    source_col: str = "embedding_pca",   # will auto-switch to normalized_col if present
    dim1_col: str = "dim1",
    dim2_col: str = "dim2",
    n_neighbors: int = 30,
    min_dist: float = 0.1,
    metric: str = "cosine",              # kept for API compat; ignored if using normalized_col
    l2_normalize: bool = True,           # ignored if using normalized_col
    random_state: int = 42,
    normalized_col: str = "embedding_norm",
    prefer_normalized: bool = True
) -> pd.DataFrame:
    """
    Run UMAP to 2D and add/overwrite 'dim1','dim2'.
    If `prefer_normalized` and `normalized_col` exists, use it and force metric='euclidean'
    (cosine-equivalent after L2 normalization). Otherwise, behaves like the original.
    """
    use_norm = prefer_normalized and (normalized_col in df.columns)

    # Pick source
    col = normalized_col if use_norm else source_col
    X = _column_of_lists_to_matrix(df, col)

    # If not using pre-normalized vectors, optionally normalize for cosine/euclidean geometry
    if not use_norm and l2_normalize and metric in ("cosine", "euclidean"):
        from sklearn.preprocessing import normalize
        X = normalize(X, norm="l2")

    # Metric choice:
    # - If using normalized vectors, euclidean == cosine (up to scaling), so force 'euclidean'
    #   to avoid any environment metric quirks.
    umap_metric = "euclidean" if use_norm else metric

    reducer = umap.UMAP(
        n_neighbors=n_neighbors,
        min_dist=min_dist,
        metric=umap_metric,
        random_state=random_state
    )
    X_2d = reducer.fit_transform(X)
    df[dim1_col] = X_2d[:, 0].astype(float)
    df[dim2_col] = X_2d[:, 1].astype(float)
    return df



In [ ]:
inputfile = 'dino_embedding' 
df = load_embeddings(f'{dino_embedding}.parquet') #Original
df = pca_reduce(df, n_components=300)         # adds 'embedding_pca'

In [ ]:
# Optional: run clustring over set of hyper parameters 
# --- define your grids (tweak as you like) ---
grid_min_cluster_size = [10, 15, 20, 30, 40, 60]
grid_min_samples      = [1, 2, 5, 10, 15, 20, None]  # None => most conservative
grid_epsilon          = [0.0, 0.02, 0.05, 0.1, 0.15, 0.2, 0.3]

results = []

combos = list(itertools.product(grid_min_cluster_size, grid_min_samples, grid_epsilon))


for mcs, ms, eps in tqdm(combos, desc="Grid search progress"):
    try:
        s_all,s,n_clusters, pct_clustered = cluster_hdbscan_silhouette(
            df,
            source_col="embedding_pca",
            min_cluster_size=mcs,
            min_samples=ms,
            cluster_selection_epsilon=eps,

        )
        # If silhouette is undefined (None), store NaN so we can sort/filter easily
        s_val = np.nan if s is None else float(s)
        s_all_val = np.nan if s_all is None else float(s_all)
    except Exception as e:
        # In case any combo fails, record as NaN (or log e)
        s_val = np.nan
        s_all_val = np.nan
                

    results.append({
        "min_cluster_size": mcs,
        "min_samples": ms if ms is not None else "None",
        "cluster_selection_epsilon": eps,
        "silhouette_include_noise": s_all_val,
        "silhouette_excl_noise": s_val,
        "number_clusters": n_clusters,
        "pct_clustered": pct_clustered
    })

df_grid = pd.DataFrame(results).sort_values(
    by="silhouette_excl_noise", ascending=False, na_position="last"
).reset_index(drop=True)


In [ ]:
df = cluster_hdbscan(df,
            source_col="embedding_pca",
            min_cluster_size=15,
            min_samples=1,
            cluster_selection_epsilon=.02
                    )
    
    #df, source_col="embedding_pca")      # adds 'cluster'
df = umap_reduce(df, source_col="embedding_pca")          # adds 'dim1','dim2'


In [ ]:
outputfile = 'embeddings_result'
df.to_parquet(f"{outputfile}.parquet")